In [35]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.utils import to_categorical


In [36]:
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Případná změna velikosti obrázku
            images.append(img)
    return np.array(images)

clean_images = load_images_from_folder('./vzorky/original')
salt_pepper_images = load_images_from_folder('./vzorky/salt_pepper_intensity_0.01')
poisson_images = load_images_from_folder('./vzorky/poisson_intensity_0.1')
speckle_images = load_images_from_folder('./vzorky/speckle_intensity_0.1')


FileNotFoundError: [Errno 2] No such file or directory: 'clean_images/'

In [28]:
X_train = []
y_train = []

for filename in os.listdir(noisy_images_dir):
    noisy_img = Image.open(os.path.join(noisy_images_dir, filename))
    clean_img = Image.open(os.path.join(clean_images_dir, filename))
    X_train.append(np.array(noisy_img))
    y_train.append(np.array(clean_img))

X_train = np.array(X_train)
y_train = np.array(y_train)

In [29]:
# Vytvoření modelu CNN pro vyčištění obrázků
height = 512
width = 512
channels = 1  # nebo 3 pro barevné obrázky

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(height, width, channels)),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(1, (3, 3), activation='sigmoid', padding='same')
])

# Kompilace modelu
model.compile(optimizer=Adam(), loss=MeanSquaredError())

# Trénink modelu
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Vyhodnocení modelu
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

# Vizualizace příkladu vyčištění
#clean_images = model.predict(X_test[:10])
clean_images = model.predict(X_train[:10])

# Vykreslení původních a vyčištěných obrázků pro porovnání
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(height, width))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(clean_images[i].reshape(height, width))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 66s 19s/step - loss: 20297.5664 - val_loss: 20975.2773
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 62s 18s/step - loss: 20408.3379 - val_loss: 20975.2754
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 65s 19s/step - loss: 20184.6504 - val_loss: 20975.2773
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 70s 20s/step - loss: 20180.3711 - val_loss: 20975.2773
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 61s 18s/step - loss: 20075.4727 - val_loss: 20975.2773
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 66s 20s/step - loss: 20635.3633 - val_loss: 20975.2773
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 61s 18s/step - loss: 20493.9609 - val_loss: 20975.2773
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 61s 18s/step - loss: 20168.6992 - val_loss: 20975.2773
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 63s 19s/step - loss: 20205.6445 - val_loss: 20975.2754
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 62s 19s/step - loss: 20189.9375 - val_loss: 20975.2734
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - loss: 20128.3516


TypeError: cannot unpack non-iterable float object